# Use the OpenAI Swarm to orchestrate multiple agents

Official Documentation: https://github.com/openai/swarm

Medium Articile Reading: https://medium.com/ai-artistry/openai-swarm-vs-langchain-langgraph-a-detailed-look-at-multi-agent-frameworks-0f978a4ca203

In [1]:
# Install Swarm in the Terminal
# pip install git+https://github.com/openai/swarm.git

In [2]:
# Import modules
import os
from dotenv import load_dotenv
from swarm import Swarm, Agent
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from pydantic import BaseModel

In [3]:
# Set your OpenAI API key
load_dotenv()
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

# A basic multi-agent orchestration example

In [4]:
# Create a Swarm client
client = Swarm()

In [5]:
# Define a function to allow the current agent to handoff to agent b
def transfer_to_agent_b():
    return agent_b

In [6]:
# Create agent A and provide it with the function to handoff to agent B
agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

# Create agent B (which does not have a function to handoff to agent A)
agent_b = Agent(
    name="Agent B",
    instructions="Only talk like a 5 year old.",
)

In [8]:
# Run the client and print the response
response = client.run(
    # Start with agent A
    agent=agent_a,
    # Provide the innitial message to agent A
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])
print()
print(response.agent.name)

I can't make phone calls or do stuff with agents. But I can chat with you! Let's talk about fun things like cartoons or superheroes! 😊

Agent B


In [9]:
# Run the client again with a different message and print the response.
# Note: agent A can decide whether to handoff to agent B or not.
response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "Fall comes and the trees are turning red. Could you provide me with a one-sentence vivid description of the falling leaves?"}],
)

print(response.messages[-1]["content"])
print()
print(response.agent.name)

The crimson leaves cascade gently from the trees, dancing through the crisp autumn air like fiery confetti celebrating the change of seasons.

Agent A


# Create two agents that can discuss a given topic to improve the solution

In [10]:
# Initialize the Swarm client
client = Swarm()

# Define agents for the discussion
agent_market = Agent(
    name="Market Expert",
    instructions="Revise the survey question further to make it more appropriate for the target customers. Provide the revised survey question with a single-sentence justification.",
)

agent_survey = Agent(
    name="Survey Expert",
    instructions="Revise the survey question further to make it scientifically sound and rigorous. Provide the revised survey question with a single-sentence justification.",
)

# Define a discussion system function
def discussion_system(agent_a, agent_b, topic, iterations):
    messages = [{"role": "user", "content": topic}]
    current_agent = agent_a

    print(f"Discussion Topic: {topic}\n")
    
    for i in range(iterations):
        # Run the current agent with the latest messages
        response = client.run(agent=current_agent, messages=messages)
        
        # Print the response from the current agent
        print(f"{current_agent.name}: {response.messages[-1]['content']}")
        print()
        
        # Add the agent's response to the conversation history
        messages.append({"role": "user", "content": response.messages[-1]["content"]})
        
        # Alternate between agents
        current_agent = agent_b if current_agent == agent_a else agent_a

# Example usage
discussion_topic = '''Survey Question: "On a scale of 1 to 10, how likely are you to recommend our product to a friend or colleague?"'''
discussion_system(agent_market, agent_survey, discussion_topic, iterations=4)

Discussion Topic: Survey Question: "On a scale of 1 to 10, how likely are you to recommend our product to a friend or colleague?"

Market Expert: Revised Survey Question: "Based on your experience with our product, how likely are you to recommend it to someone you know?"

Justification: This revision personalizes the question by emphasizing the respondent's experience with the product, making it more relevant and encouraging a thoughtful response.

Survey Expert: Revised Survey Question: "Considering your recent usage, how would you rate your likelihood of recommending our product to a friend or colleague on a scale from 1 (very unlikely) to 10 (extremely likely)?"

Justification: This revision specifies the timeframe ("recent usage") to focus the respondent's assessment, retains the numerical scale for quantitative analysis, and clarifies the scale's meaning to ensure consistent understanding and responses.

Market Expert: Revised Survey Question: "Based on your recent experience, how

# Create two agents that debate each other based on a topic given by the judge, and the judge decides which agent wins

In [11]:
# Initialize the Swarm client
client = Swarm()

# Define the agents
debater_a = Agent(
    name="Debater A",
    instructions="Argue in favor of the provided statement. Use up to 3 sentences in your speech.",
)

debater_b = Agent(
    name="Debater B",
    instructions="Argue against the provided statement. Use up to 3 sentences in your speech.",
)

judge = Agent(
    name="Judge",
    instructions="Listen to both sides and declare a winner with justification.",
)

# Define the debate system function
def debate_system(judge, debater_a, debater_b, topic, rounds=3):
    print(f"Debate Topic: {topic}\n")

    messages = [{"role": "user", "content": topic}]
    
    # Run the debate for the specified number of rounds
    for round_num in range(1, rounds + 1):
        print(f"--- Round {round_num} ---")
        
        # Debater A's turn
        response_a = client.run(agent=debater_a, messages=messages)
        print(f"{debater_a.name}: {response_a.messages[-1]['content']}\n")
        messages.append({"role": "user", "content": response_a.messages[-1]["content"]})
        
        # Debater B's turn
        response_b = client.run(agent=debater_b, messages=messages)
        print(f"{debater_b.name}: {response_b.messages[-1]['content']}\n")
        messages.append({"role": "user", "content": response_b.messages[-1]["content"]})
    
    # After the debate rounds, the judge provides comments and picks a winner
    print("--- Judge's Evaluation ---")
    judge_response = client.run(agent=judge, messages=messages + [{"role": "user", "content": "Please evaluate and decide a winner."}])
    
    # Print judge's final response with comments and winner
    print(f"{judge.name}: {judge_response.messages[-1]['content']}\n")

# Example usage
debate_topic = "Highschool students should have PE every school day."
debate_system(judge, debater_a, debater_b, topic=debate_topic, rounds=3)

Debate Topic: Highschool students should have PE every school day.

--- Round 1 ---
Debater A: High school students should have PE every school day as it promotes consistent physical activity, which is crucial for maintaining physical health, combating obesity, and reducing stress levels. Regular physical education classes also help students develop life-long healthy habits, improve mental health, and enhance overall academic performance by increasing focus and concentration. Furthermore, daily PE classes provide essential opportunities for social interaction and teamwork, which are key skills for personal and professional success.

Debater B: Mandating PE every school day might lead to an imbalance in the academic curriculum, taking away precious time from core subjects where students need to build crucial knowledge and skills for college and future careers. Not all students are athletically inclined, and imposing daily physical education could increase stress and anxiety, detracting 

# Design two agents and a supervisor who chooses the most suitable agent to run a task

In [12]:
# Define a function to report the weather
def get_weather(city: str):
    return f"The weather in {city} is always 30°C."

# Define a function to report the air quality
def get_air_quality(city: str):
    return f"The air quality in {city} is 'Good' with an AQI of 42."

In [13]:
# Define a function to route requests to the appropriate agent.
# Note: The context_variables dictionary contains the last user message.
def route_request(context_variables):
    # Check if the user message contains "weather" or "air quality".
    user_message = context_variables.get("last_user_message", "")
    
    # Route the request to the appropriate agent based on the user message.
    if "weather" in user_message.lower():
        return weather_agent
    elif "air quality" in user_message.lower():
        return air_quality_agent
    else:
        return "I'm sorry, I don't understand your request."

In [14]:
# Create the weather agent
weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a weather assistant. Provide the weather information when asked.",
    functions=[get_weather] # Provide the get_weather function to the agent
)

# Create the air quality agent
air_quality_agent = Agent(
    name="Air Quality Agent",
    instructions="You are an air quality assistant. Provide the air quality information when asked.",
    functions=[get_air_quality] # Provide the get_air_quality function to the agent
)

# Create the supervisor agent
supervisor_agent = Agent(
    name="Supervisor Agent",
    instructions="You are the supervisor. Determine whether the user's request is about weather or air quality, and transfer them to the appropriate agent by calling the 'route_request' function.",
    functions=[route_request] # Provide the route_request function to the agent
)

In [15]:
# Provide a user message and construct the context_variables dictionary
user_message = "What's the weather like in Paris?"
context_variables = {"last_user_message": user_message}

In [16]:
# Run the client and print the response
response = client.run(
    # Start with the supervisor agent
    agent=supervisor_agent,
    # Provide the innitial message to the supervisor agent
    messages=[{"role": "user", "content": user_message}],
    # Provide the context variables to the supervisor agent
    context_variables=context_variables
)

print(response.messages[-1]["content"])

The current weather in Paris is 30°C.


In [17]:
# Try a different user message
user_message = "How is the air quality in New York City?"
context_variables = {"last_user_message": user_message}

response = client.run(
    agent=supervisor_agent,
    messages=[{"role": "user", "content": user_message}],
    context_variables=context_variables
)

print(response.messages[-1]["content"])

The air quality in New York City is currently 'Good' with an Air Quality Index (AQI) of 42.


In [18]:
# However, because the route_request is hardcoded, it can be stubborn.
user_message = "How is the quality of air in New York City?"
context_variables = {"last_user_message": user_message}

response = client.run(
    agent=supervisor_agent,
    messages=[{"role": "user", "content": user_message}],
    context_variables=context_variables
)

print(response.messages[-1]["content"])

It seems there is an issue with transferring your request. Please try asking again later, or check a reliable source for air quality information in New York City directly.


We can allow the supervisor agent to route the agents based on semantic understanding of the task by making changes to the route_request function.

In [19]:
# Create a template using the pydantic basemodel
class ChooseAgent(BaseModel):
    '''Output an agent name based on the user's question.'''
    agent_name: str

In [20]:
# Build a chat prompt template
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', 
    '''
    You will determine whether the user's request is about weather or air quality. 
    If the request is about weather, you will output "weather_agent"; if the request is about air quality,
    you will output "air_quality_agent"; and if the request is neither about weather nor about air quality, you return "None".
    The user's request is: {request}.
    '''),
])

# Call a chatmodel
llm = ChatOpenAI()

# Build the chain
choose_agent_chain = template | llm.with_structured_output(ChooseAgent)

# User's request
request = "How is the quality of air in New York City?"

# Invoke the chain
result = choose_agent_chain.invoke({
    "request": request,
})

# Print the result
print(result)

agent_name='air_quality_agent'


In [21]:
# Based upon the above choose_agent_chain, we can create a more flexible route request fuction.
def route_request_semantic(user_message): 
    # Invoke the chain to determine the agent to use.
    result = choose_agent_chain.invoke({
    "request": user_message,
    })

    # Route the request to the appropriate agent based on the user message.
    if "weather" in result.agent_name.lower():
        return weather_agent
    elif "air" in result.agent_name.lower():
        return air_quality_agent
    else:
        return "I'm sorry, I don't understand your request."


# Re-create the supervisor agent
supervisor_agent_semantic = Agent(
    name="Supervisor Agent",
    instructions="You are the supervisor. Determine whether the user's request is about weather or air quality, and transfer them to the appropriate agent by calling the 'route_request' function.",
    functions=[route_request_semantic]
)

In [22]:
# Try the enhanced route request function
user_message = "How is the quality of air in New York City?"
context_variables = {"last_user_message": user_message}

response = client.run(
    agent=supervisor_agent_semantic,
    messages=[{"role": "user", "content": user_message}],
    context_variables=context_variables
)

print(response.messages[-1]["content"])

The air quality in New York City is 'Good' with an AQI (Air Quality Index) of 42.
